In [3]:
import tensorflow as tf
import numpy as np
import cv2

/home/kiizuka/.venv/python3-tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kiizuka/.venv/python3-tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kiizuka/.venv/python3-tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kiizuka/.venv/python3-tf/lib/python3.6/site-packages

In [12]:
img_path = './italia.jpg'
img = cv2.imread(img_path)
print(img.shape)
img = cv2.resize(img, (80, 53))
print(img.shape)
# print(img)
# for i in range(img.shape[0]):
#     for j in range(img.shape[1]):
#         for k in range(img.shape[2]):
#             print(img[i][j])
# print(img[10][39][1])
trans_img = img.transpose([2, 0, 1])
print(trans_img.shape)
# print(trans_img[1][10][39])
for i in range(trans_img.shape[0]):
    for j in range(trans_img.shape[1]):
        for k in range(trans_img.shape[2]):
            if(trans_img[i][j][k] != img[j][k][i]):
                print("The wrong transpose data")
                print("CH::{}, H::{}, W::{}".format(i, j, k))

(534, 800, 3)
(53, 80, 3)
(3, 53, 80)


## Compare weight data (in npy file vs in onnx data)

In [23]:
def load_weights(op_name, weights_path='./bvlc_alexnet.npy'):
    weights_dict = np.load(weights_path, allow_pickle=True, encoding='bytes').item()
    return weights_dict[op_name]

In [24]:
tf_weight = load_weights('conv1')[0].transpose([3,2,0,1])
tf_bias = load_weights('conv1')[1]
onnx_weight = np.load('onnx_conv1_w_0', allow_pickle=True, encoding='bytes')
onnx_bias = np.load('onnx_conv1_b_0', allow_pickle=True, encoding='bytes')

In [25]:
if(tf_weight.shape != onnx_weight.shape):
    print("The size of weight is wrong")
    print("TF_W:: ", tf_weight.shape)
    print("ONNX_W:: ", onnx_weight.shape)
if(tf_bias.shape != onnx_bias.shape):
    print("The size of bias is wrong")
if(tf_weight.dtype != onnx_weight.dtype):
    print("The data type of weight is wrong")
if(tf_bias.dtype != onnx_bias.dtype):
    print("The data type of bias is wrong")

In [26]:
onnx_w_flat = onnx_weight.flatten()
tf_w_flat = tf_weight.flatten()
print(np.array_equal(tf_w_flat, onnx_w_flat))
      
onnx_b_flat = onnx_bias.flatten()
tf_b_flat = tf_bias.flatten()
print(np.array_equal(tf_b_flat, onnx_b_flat))

True
True


### Check C++ header value and numpy array

### Error check function

In [28]:
def error_check(arr1, arr2):
    if np.array_equal(arr1, arr2):
        print("Two array are equal")
        return
    if arr1.shape != arr2.shape:
        print("Two array size are different")
        exit()
    errors = []
    for i, e in enumerate(arr1):
        error = abs(arr1[i] - arr2[i])
        errors.append(error)
    errors = np.array(errors, dtype=np.float32)
    return errors

In [29]:
header_bias = np.loadtxt('fpga_b.txt', dtype = np.float32)
header_weight = np.loadtxt('fpga_w.txt', dtype = np.float32)
# Check bias data
bias_errors = error_check(tf_b_flat, header_bias)
max_b_error_index = bias_errors.argmax()
print([max_b_error_index, bias_errors[max_b_error_index], header_bias[max_b_error_index], tf_b_flat[max_b_error_index]])
# Check weight data
weight_errors = error_check(tf_w_flat, header_weight)
max_w_error_index = weight_errors.argmax()
print([max_w_error_index, weight_errors[max_w_error_index]])

[1, 4.7683716e-07, 0.283837, 0.2838365]
[701, 5.066395e-07]


In [30]:
header_string = np.array2string(header_bias,separator=',')
print(header_string)

[-0.346397 , 0.283837 ,-0.499682 ,-0.236491 ,-0.568887 ,-0.643664 ,
 -0.692477 ,-0.495555 ,-0.232274 , 0.255125 ,-0.264854 ,-0.413465 ,
 -0.434615 ,-0.700091 ,-0.474967 ,-0.475487 ,-0.6968   ,-0.465743 ,
 -0.527414 ,-0.743593 ,-0.691516 ,-0.505254 ,-0.26251  ,-0.651338 ,
 -0.411121 ,-0.599743 ,-0.68655  ,-0.501677 ,-0.722321 ,-0.606828 ,
 -0.601712 ,-0.49667  ,-0.561121 ,-0.483873 ,-0.611091 ,-0.54242  ,
 -0.569912 ,-0.436019 ,-0.395642 ,-0.371173 , 0.036627 ,-0.229277 ,
  0.0961724,-0.480799 , 0.0201771,-0.772401 ,-0.63783  ,-0.592662 ,
 -0.472413 ,-0.515198 ,-0.715582 ,-0.263552 ,-0.629393 ,-0.323797 ,
 -0.655959 ,-0.0885214,-0.635241 ,-0.588488 ,-0.552137 ,-0.326814 ,
 -0.484541 ,-0.613501 ,-0.623194 ,-0.715386 ,-0.746007 ,-0.591131 ,
 -0.382403 ,-0.315873 ,-0.646048 ,-0.569708 ,-0.675176 ,-0.708902 ,
 -0.410006 ,-0.504767 ,-0.558701 ,-0.628178 ,-0.0866769,-0.510795 ,
  0.172265 ,-0.547054 ,-0.559364 ,-0.551644 ,-0.181644 ,-0.0146798,
 -0.299411 ,-0.382882 ,-0.0946849,-0.0959714,-0.

## Compare resized image between np.uint8 and np.float32

In [37]:
img = cv2.imread('./images/zebra.jpeg')
uint8_img = cv2.resize(img, (227, 227))
fp_img = cv2.resize(img.astype(np.float32), (227, 227))

def error_check(arr1, arr2):
    if np.array_equal(arr1, arr2):
        print("Two array are equal")
        return
    if arr1.shape != arr2.shape:
        print("Two array size are different")
        exit()
    errors = []
    for i, e in enumerate(arr1):
        error = abs(arr1[i] - arr2[i])
        errors.append(error)
    return errors

errors = error_check(uint8_img, fp_img)

## Compare the result of convolution in 1st layer of AlexNet(Tensorflow vs my fpga C simulation)

In [56]:
# fpga_result = np.loadtxt('fpga_conv_result', dtype = np.float32)
# tf_result = np.loadtxt('tf_conv1_result_use_fpga.txt', dtype = np.float32)
fpga_result = np.fromfile('fpga_conv_result.bin', dtype = np.float32)
tf_result = np.fromfile('tf_conv1_result.bin', dtype = np.float32)

In [57]:
if(fpga_result.shape != tf_result.shape):
    print("The size of weight is wrong")
    print("FPGA:: ", fpga_result.shape)
    print("TF:: ", tf_result.shape)
if(fpga_result.dtype != tf_result.dtype):
    print("The data type of bias is wrong")
if not np.array_equal(fpga_result, tf_result):
    print("error is somewhere...")
print(fpga_result.shape)

False
(290400,)


In [58]:
errors = []
for i, e in enumerate(fpga_result):
    error = abs(tf_result[i] - fpga_result[i])
    errors.append(error)

In [60]:
error_ratio = [abs(e[0]/e[1]) for e in zip(errors, tf_result)]

In [61]:
error_ratio = np.array(error_ratio, dtype=np.float32)
mi = error_ratio.argmax()
print([mi, errors[mi], error_ratio[mi], tf_result[mi], fpga_result[mi]])

[93737, 5.9366226e-05, 0.9188192, 6.4611435e-05, 5.2452087e-06]


In [65]:
cnt = 0
for i, err in enumerate(error_ratio):
    if err > 1.0e-01:
        cnt = cnt+1
print(cnt)

2
